# Question

왜 Q-learning 은 off-policy 인데 importance sampling 을 사용하지 않는가?

### Sarsa

![sarsa](sarsa.png)

#### python pesudo code

on-policy 이므로 target policy = behavior policy = policy 로 두자.

```py
policy = e_greedy() # e-soft policy
for episode in episodes:
    # states[0]: initial state
    S = states[0]
    # 초기상태 S0 에서 Q 에 따라 액션을 선택
    A = policy(S, Q)
    while not terminal(S):
        # state S 에서 action A 를 선택하면 environment 에서 다음 상태 S' 과 reward R 을 리턴
        R, S` = env(S, A)
        A` = policy(S`, Q)
        
        # update (gamma: discount factor)
        Q[S, A] = Q[S, A] + alpha*(R + gamma*Q[S`, A`] - Q[S, A])
        
        # go to the next step
        S = S`
        A = A`
```

### Q-learning

![q-learning](q-learning.png)

#### python pseudo code

```py
target_policy = greedy()
behavior_policy = e_greedy()
for episode in episodes:
    S = states[0]
    while not terminal(S):
        A = behavior_policy(S, Q) # e-greedy
        R, S` = env(S, A)
        A` = target_policy(S`, Q) # greedy
        
        # update
        Q[S, A] = Q[S, A] + alpha*(R + gamma*Q[S`, A`] - Q[S, A])
        
        # go to the next state
        S = S`
```

여기서 target_policy 가 greedy 이므로 그냥 식에 집어넣어 줄 수도 있다 (책 버전; in-place):

```py
behavior_policy = e_greedy()
for episode in episodes:
    S = states[0]
    while not terminal(S):
        A = behavior_policy(S, Q) # e-greedy
        R, S` = env(S, A)
        
        # update
        # max(Q[S`, :]) == greedy(S`, Q).
        Q[S, A] = Q[S, A] + alpha*(R + gamma*max(Q[S`, :]) - Q[S, A])
        
        # go to the next state
        S = S`
```

## Why do not use importance sampling?

왜 안쓸까?

- 일단 왜 쓰는지를 생각해보자.
- MC 에서 behavior policy 는 e-greedy 고, target policy 는 greedy 일 때, behavior policy 로 구한 return 으로 target policy 를 업데이트 해 주기 위해 사용
    - importance sampling 을 통해 behavior policy 에 대한 return $G_b$ 를 target policy 에 대한 return $G_\pi$ 로 바꿈!

### Q-learning

- Q-learning 에서는 일단 return G 를 직접적으로 다루지 않음. 
- 하지만 $v_\pi(s)=\mathbb E_\pi [G_t] = \mathbb E_\pi[R_{t+1}+\gamma v_\pi(S_{t+1})]$.
- 즉 max Q(S', a) 를 하게 되면 이게 target policy pi 에 대한 expected return 이므로, 여기다가 importance sampling ratio 를 곱해줘야 할 것 같은데.

왜 안하지?

깨달았다!

- MC 에서는 업데이트를 return G 로 함. 따라서 이 G 가 정확해야 한다. 
    - 그래서 behavior policy 로 구한 $G_b$ 를 target policy 에 대한 return $G_\pi$ 로 변환하는 작업을 importance sampling 으로 해줌.
- 하지만 TD (Q-learning) 에서는 업데이트를 return G 가 아니라 R + Q(S', A') 으로 함.
- 여기서 R 은 정확하기 때문에 Q(S', A') 만 정확하면 됨.
    - G 와 마찬가지로 결국 이 Q 가 $Q_b$ 냐 $Q_\pi$ 냐의 문제. 
    - 우리는 target policy pi 로 Q 를 업데이트하고 있으므로 학습되는 Q 는 $Q_\pi$ 가 된다.
    - 즉 Q 는 이미 $Q_\pi$ 이므로 굳이 importance sampling 을 통해 변환해 줄 필요가 없는 것!

Summary:

- MC 에서는 $G_b$ 를 $G_\pi$ 로 변환하기 위해 importance sampling 을 사용
- TD 에서는 이 return G 를 쓰는 것이 아니라 Q 를 씀
- 따라서 MC 와 마찬가지로 $Q_b$ 로 $Q_\pi$ 를 업데이트 하고 싶다면 importance sampling 으로 변환해줘야 하나, Q-learning 에서는 처음부터 $Q_\pi$ 를 학습하므로 변환해줄 필요가 없음

사족:

- sarsa 에서 behavior policy 로 A' 을 구한 다음에 Q 를 업데이트 해줄 때 여기서 importance sampling 을 적용해서 업데이트 해주는 방식으로 사용할수는 있을 듯!
    - 다만 이렇게 하려면 $Q_b$ 와 $Q_\pi$ 두 Q table 을 운용해야 할 듯